In [9]:
import smtplib
import os
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from password import username, password


YOUTUBE_TRENDING_URL = 'https://www.youtube.com/feed/trending'



def get_driver():
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-dev-shm-usage')
    s = Service('D:\python\chromedriver.exe')
    driver = webdriver.Chrome(service=s,options=chrome_options)
    return driver

d = get_driver()

def get_videos(driver):
    VIDEO_DIV_TAG = 'ytd-video-renderer'
    driver.get(YOUTUBE_TRENDING_URL)
    videos = driver.find_elements(By.TAG_NAME, VIDEO_DIV_TAG)
    return videos

def parse_video(video):
    title_tag = video.find_element(By.ID,'video-title')
    title_text = video.find_element(By.ID, 'video-title').text
    url = title_tag.get_attribute('href')
    thumbnail_tag = video.find_element(By.TAG_NAME,'img')
    thumbnail_url = thumbnail_tag.get_attribute('src')
    channel_text = video.find_element(By.CLASS_NAME,'ytd-channel-name').text
    return {
        'Title': title_text,
        'Url': thumbnail_url,
        'Channel Name': channel_text
        }


def send_email(body):
    
    server_ssl = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server_ssl.ehlo()   

    emailfrom = 'yb4coding@gmail.com'
    emailto = 'yb4coding@gmail.com'
    fileToSend = "final info.csv"

    username = username
    password = password
    msg = MIMEMultipart()
    msg["From"] = emailfrom
    msg["To"] = emailto
    msg["Subject"] = "Spreadsheet of apple mobiles-flipkart"
        

    ctype, encoding = mimetypes.guess_type(fileToSend)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"

    maintype, subtype = ctype.split("/", 1)

    if maintype == "text":
        fp = open(fileToSend)
        # Note: we should handle calculating the charset
        attachment = MIMEText(fp.read(), _subtype=subtype)
        fp.close()
    else:
        fp = open(fileToSend, "rb")
        attachment = MIMEBase(maintype, subtype)
        attachment.set_payload(fp.read())
        fp.close()
        encoders.encode_base64(attachment)
        attachment.add_header("Content-Disposition", "attachment", filename=fileToSend)
        msg.attach(attachment)

        server = smtplib.SMTP("smtp.gmail.com:587")
        server.starttls()
        server.login(username,password)
        server.sendmail(emailfrom, emailto, msg.as_string())
        server.quit()


if __name__ == "__main__":
    print('Creating driver')
    driver = get_driver()

    print('Fetching trending videos')
    videos = get_videos(driver)

    print(f'Found {len(videos)} videos')

    print('Parsing top 10 videos')
    videos_data = [parse_video(video) for video in videos[:10]]

    print('Save the data to a CSV')
    videos_df = pd.DataFrame(videos_data)
    # print(videos_df)
    csv = videos_df.to_csv('trending.csv', index=None)

    print("Send the results over email")
    body = csv
    send_email(body)

    print('Finished.')

    
    
    

Creating driver
Fetching trending videos
Found 97 videos
Parsing top 10 videos
Save the data to a CSV
Send the results over email
Finished.
